Установка модели spaCy (выполните в отдельной ячейке при первом запуске)

Windows/macOS/Linux:

```python -m spacy download ru_core_news_sm```

In [28]:
import pandas as pd
import json
import re
import spacy
import nltk
from pathlib import Path
from nltk.corpus import stopwords


In [29]:
# Загрузка моделей и ресурсов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))
nlp = spacy.load("ru_core_news_sm")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maitre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# Путь к последнему JSON-файлу с данными
raw_data_dir = Path.cwd().parent / "data" / "raw"
json_files = sorted(raw_data_dir.glob("ria_news_*.json"), reverse=True)
latest_file = json_files[0] if json_files else None

if not latest_file:
    raise FileNotFoundError("Нет доступных JSON-файлов в data/raw")

print(f"Загружается файл: {latest_file.name}")


Загружается файл: ria_news_20250429_164325.json


In [31]:
# Загрузка JSON
with open(latest_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)


In [32]:
# Преобразование в плоский DataFrame
rows = []
for category, articles in raw_data.items():
    for url, content in articles.items():
        tags = content.get("tags") if content.get("tags") else []
        tags = [t for t in tags if t.strip() != ""]
        rows.append({
            "category": category,
            "url": url,
            "title": content.get("title"),
            "views": content.get("views"),
            "tags": tags
        })

df = pd.DataFrame(rows)
print(f"Всего записей: {len(df)}")


Всего записей: 1200


In [33]:
# Очистка данных
df.dropna(subset=["title", "views"], inplace=True)
df.drop_duplicates(subset="url", inplace=True)
df["title"] = df["title"].str.strip().str.lower()
df["views"] = pd.to_numeric(df["views"], errors="coerce")
df["tags"] = df["tags"].apply(lambda tags: [t for t in tags if t.strip()])


In [34]:
# --- Обработка текста ---
def preprocess_title(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if token.is_alpha]
    tokens_no_stop = [t for t in tokens if t.lower() not in stop_words]
    lemmas = [token.lemma_ for token in doc if token.is_alpha and token.lemma_.lower() not in stop_words]
    ents = [(ent.text, ent.label_) for ent in doc.ents]
    return {
        "clean": " ".join(tokens_no_stop),
        "tokens": tokens_no_stop,
        "lemmas": lemmas,
        "entities": ents
    }

processed = df["title"].apply(preprocess_title)
df["title_clean"] = processed.map(lambda x: x["clean"])
df["title_tokens"] = processed.map(lambda x: x["tokens"])
df["title_lemmas"] = processed.map(lambda x: x["lemmas"])
df["named_entities"] = processed.map(lambda x: x["entities"])


In [26]:
# Сохраняем чистые данные
output_path = Path.cwd().parent / "data" / "processed"
output_path.mkdir(parents=True, exist_ok=True)
clean_file = output_path / "clean_articles.csv"
df.to_csv(clean_file, index=False, encoding="utf-8")

print(f"Готово. Сохранено в: {clean_file}")
df.head()


Готово. Сохранено в: D:\Coding\hhiissookkaax_project1\data\processed\clean_articles.csv


,category,url,title,views,tags,title_clean,title_tokens,title_lemmas,named_entities
0,Политика,https://rsport.ria.ru/20250429/stupak-20141064...,вяльбе поддержала участие ступак в партийных в...,25,"[Политика, Лыжные гонки, Республика Коми, Сосн...",вяльбе поддержала участие ступак партийных выб...,"[вяльбе, поддержала, участие, ступак, партийны...","[вяльбе, поддержать, участие, ступак, партийны...","[(вяльбе, PER)]"
1,Политика,https://ria.ru/20250429/putin-2014072360.html,путин прилетел в волгоград,1825,"[Политика, Волгоград, Россия, Владимир Путин, ...",путин прилетел волгоград,"[путин, прилетел, волгоград]","[путин, прилететь, волгоград]","[(путин, PER), (волгоград, LOC)]"
2,Политика,https://ria.ru/20250429/volodin-2014019151.html,володин назвал защиту исторической правды одни...,263,"[Политика, Белоруссия, Волгоград, Владимир Путин]",володин назвал защиту исторической правды одни...,"[володин, назвал, защиту, исторической, правды...","[володин, назвать, защита, исторический, правд...","[(володин, PER), (сг, ORG)]"
3,Политика,https://ria.ru/20250429/peskov-2014002713.html,песков высказался о легитимности зеленского на...,9770,"[Политика, Россия, Дмитрий Песков, Владимир Зе...",песков высказался легитимности зеленского пост...,"[песков, высказался, легитимности, зеленского,...","[песков, высказаться, легитимность, зеленского...","[(песков, PER)]"
4,Политика,https://ria.ru/20250429/klishas-2013972100.html,сенатор клишас шуткой отреагировал на слова во...,11183,"[Политика, Госдума РФ, Андрей Клишас, Вячеслав...",сенатор клишас шуткой отреагировал слова волод...,"[сенатор, клишас, шуткой, отреагировал, слова,...","[сенатор, клишас, шутка, отреагировать, слово,...","[(клишас шуткой, PER)]"
